<a href="https://colab.research.google.com/github/PDNow-Research/PDNow/blob/main/HandPD/NeuralNets/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import sys
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from PIL import Image
import Image
print(Image.__file__)

In [ ]:
# when preprocessing resize to (100, 100) --> https://www.holisticseo.digital/python-seo/resize-image/

In [ ]:
WIDTH = 100
HEIGHT = 100

In [ ]:
train_data = ImageDataGenerator()
val_data = ImageDataGenerator()
test_data = ImageDataGenerator()

train_generator = train_data.flow_from_directory(
    train_dir, 
    target_size = (WIDTH, HEIGHT), 
    batch_size = 4, 
    class_mode = 'binary' 
)

val_generator = val_data.flow_from_directory(
    val_dir, 
    target_size = (WIDTH, HEIGHT), 
    batch_size = 4, 
    class_mode = 'binary' 
)

test_generator = test_data.flow_from_directory(
    test_dir, 
    target_size = (WIDTH, HEIGHT), 
    batch_size = 4, 
    class_mode = 'binary' 
)

In [ ]:
model = tf.keras.models.Sequential([
    #conv net part
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)), #3 is for rgb color channels
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    #dnn part
    tf.keras.layers.Flatten(),
   
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),

    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

In [ ]:
class callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
    #creating a callback function that activates if the accuracy is greater than 60%
        if (logs.get('accuracy') > 0.96):
          print("Stopping Training")
          self.model.stop_training = True
            
callbacks = callback()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr = 0.0001),
              metrics=['accuracy'])

In [ ]:
#training model after loading data in using an image generator
history = model.fit (
    #training part
    train_generator,
    steps_per_epoch = 4, 
    epochs = 15,
    
    #testing part
    validation_data = val_generator,
    validation_steps = 4,
    callbacks =[callbacks],
    verbose = 1
)